In [4]:
### given a few pairs of cointegrated coins, use Kalman filtering to calculate the moving slope and intercept between the two coins
# use moving slope and intercepts to calculate spread functions for the pairs
# use the spread functions to calculate points in the time series where spread is 1.96 SD above or below mean
# optional: use this strategy starting with imaginary 1k to plot equity over time. ###

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement statsmodel.api (from versions: none)
ERROR: No matching distribution found for statsmodel.api

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# to get started right now, use stock prices from research paper as an example, insert crypto pairs later.
import tc_user_info
import requests
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from pykalman import KalmanFilter
from sklearn.linear_model import LinearRegression
plt.style.use('seaborn-whitegrid')

In [14]:
def get_historical_ohlcv_data(time_start, asset, num_items):
    """ Get historical OHLCV data from coinapi.io """
    url = 'https://rest.coinapi.io/v1/exchangerate/' + asset + '/USD/history?period_id=1MIN&time_start=' + time_start + '&limit=' + str(num_items)
    headers = {'X-CoinAPI-Key' : tc_user_info.COINAPIKEY}
    response = requests.get(url, headers=headers)
    data = response.json()
    df = pd.DataFrame(data)
    df['time_close'] = pd.to_datetime(df['time_close'])
    df = df.set_index('time_close')
    return df

In [15]:
btc_data = get_historical_ohlcv_data("2022-11-21T00:00:00", "BTC", 1000)
btc_data

,time_period_start,time_period_end,time_open,rate_open,rate_high,rate_low,rate_close
time_close,,,,,,,
2022-11-21 00:00:00+00:00,2022-11-21T00:00:00.0000000Z,2022-11-21T00:01:00.0000000Z,2022-11-21T00:00:00.0000000Z,16256.189979,16256.189979,16256.189979,16256.189979
2022-11-21 00:01:00+00:00,2022-11-21T00:01:00.0000000Z,2022-11-21T00:02:00.0000000Z,2022-11-21T00:01:00.0000000Z,16264.915625,16264.915625,16264.915625,16264.915625
2022-11-21 00:02:00+00:00,2022-11-21T00:02:00.0000000Z,2022-11-21T00:03:00.0000000Z,2022-11-21T00:02:00.0000000Z,16253.132224,16253.132224,16253.132224,16253.132224
2022-11-21 00:03:00+00:00,2022-11-21T00:03:00.0000000Z,2022-11-21T00:04:00.0000000Z,2022-11-21T00:03:00.0000000Z,16245.517644,16245.517644,16245.517644,16245.517644
2022-11-21 00:04:00+00:00,2022-11-21T00:04:00.0000000Z,2022-11-21T00:05:00.0000000Z,2022-11-21T00:04:00.0000000Z,16234.319268,16234.319268,16234.319268,16234.319268
...,...,...,...,...,...,...,...
2022-11-21 16:35:00+00:00,2022-11-21T16:35:00.0000000Z,2022-11-21T16:36:00.0000000Z,2022-11-21T16:35:00.0000000Z,15941.185836,15941.185836,15941.185836,15941.185836
2022-11-21 16:36:00+00:00,2022-11-21T16:36:00.0000000Z,2022-11-21T16:37:00.0000000Z,2022-11-21T16:36:00.0000000Z,15970.331276,15970.331276,15970.331276,15970.331276
2022-11-21 16:37:00+00:00,2022-11-21T16:37:00.0000000Z,2022-11-21T16:38:00.0000000Z,2022-11-21T16:37:00.0000000Z,15960.543539,15960.543539,15960.543539,15960.543539
